In [12]:
import requests
import zipfile
from pathlib import Path

import torch
from torch import nn

import torchvision
from torchvision import transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

print(torch.__version__)
print(torchvision.__version__)

2.0.1
0.15.2


## Dataset

In [13]:
data_path = Path("data/")
image_path = data_path / "images"

if image_path.is_dir():
    print("Directory exists.")
else:
    print("{image_path} path does not exist. Creating...")
    image_path.mkdir(parents=True, exist_ok=True)

with open(image_path / "images.zip", "wb") as f:
    print("Downloading images...")
    requests = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Images downloaded, writing to file...")
    f.write(requests.content)

Directory exists.
Images downloaded, writing to file...


In [14]:
## unzip the images
print("Unzipping images...")
with zipfile.ZipFile(image_path / "images.zip", "r") as zip_ref:
    zip_ref.extractall(image_path)
    print("Images unzipped.")

Unzipping images...
Images unzipped.
